In [ ]:
# %% Imports
import numpy as np, pandas as pd, torch, gpytorch, json, random, math
from pathlib import Path
from functools import lru_cache
from tqdm import tqdm
from os import sys
sys.path.append('../src/')

from esm.models.esmc import ESMC
from esm.sdk.api import ESMProtein, LogitsConfig

from embeddings.esmc_encoder import embed_sequences, embed_single
from developability.biophi_api import score_sequences
from models.mf_gp_model import MultiFidelityGP
from models.gp_model import DevelopabilityGP
from acquisition.acq import expected_improvement, upper_confidence_bound, thompson_sampling
from acquisition.mutate_seq import hill_climb, genetic_algorithm, gibbs_sampling
from utils.io import make_embedding_lookup

# %% Config & constants
from dataclasses import dataclass

FID_MAP = {"y_low": 0.3, "y_medium": 0.7, "y_high": 0.95}

@dataclass
class PipelineConfig:
    acq: str = "ei"
    seq_opt: str = "gs"
    seq_proposals: int | None = None
    dev_weight: float = 0.3
    xi: float = 0.01
    kappa: float = 2.0
    n_iter: int = 1
    batch_k: int = 1
    embed_components: int = 64
    bounds_scale: float = 1.0

cfg = PipelineConfig()

In [14]:
# %% Load & preview raw sequences
DATA_PATH = Path('../data/raw/cd98_64_seq.json')
df = pd.read_json(DATA_PATH, lines=True)
print(f'Loaded {len(df):,} sequences')
df.head()


Loaded 64 sequences


,sequence,y_low,y_medium,y_high
0,EVQLVESGGGLVQPGGSLRLSCAASGFTFKSYAMDWVRQAPGKQRE...,-267.424127,-267.561455,-267.465345
1,EVQLVESGGGLVQPGGSLRLSCAASGFTFKSYAMDWVRQAPGKQRE...,-267.612351,-267.202450,-267.164703
2,EVQLVESGGGLVQPGGSLRLSCAASGFTFKSYAMDWVRQAPGKQRE...,-270.539386,-270.320750,-270.202944
3,EVQLVESGGGLVQPGGSLRLSCAASGFTFKSYAMDWVRQAPGKQRE...,-274.479366,-274.162201,-274.071817
4,EVQLVESGGGLVQPGGSLRLSCAASGFTFKSYAMDWVRQAPGKQRE...,-274.999357,-274.873697,-274.816321


In [15]:
# %% Load or compute PCA embeddings (column: pca_embed)
EMB_FILE = Path('../data/interim/cd98_64_embed.jsonl')
embed_col = "pca_embed"

# 1) Read whatever interim JSON you already have (might contain full rows)
if EMB_FILE.exists():
    df = pd.read_json(EMB_FILE, lines=True)
else:
    # start from your raw input
    df = pd.read_json('../data/raw/cd98_64_seq.json', lines=True)

# 2) If column exists, only fill NAs; else compute for all
if embed_col in df.columns:
    mask = df[embed_col].isna()
else:
    df[embed_col] = np.nan
    mask = pd.Series(True, index=df.index)

if mask.any():
    # train PCA on _all_ sequences once
    full_seqs = df[['sequence']].copy()
    df_full = embed_sequences(full_seqs, n_components=cfg.embed_components)
    # fill missing slots
    df.loc[mask, embed_col] = df_full.loc[mask, embed_col].values

# 3) save the full DataFrame—including all original cols + pca_embed
df.to_json(EMB_FILE, orient='records', lines=True)
print(f"Embeddings ready: {df[embed_col].notna().sum()} / {len(df)} sequences")


Embeddings ready: 64 / 64 sequences


In [16]:
# %% Load or compute developability scores (column: dev_score)
DEV_FILE = Path('../data/interim/cd98_64_biophi.jsonl')
dev_col = "dev_score"

# 1) Load interim if present, else raw
if DEV_FILE.exists():
    df = pd.read_json(DEV_FILE, lines=True)
else:
    df = pd.read_json('../data/interim/cd98_64_embed.jsonl', lines=True)

# 2) Fill or create
if dev_col in df.columns:
    mask = df[dev_col].isna()
else:
    df[dev_col] = np.nan
    mask = pd.Series(True, index=df.index)

if mask.any():
    scores = score_sequences(df.loc[mask, 'sequence'].tolist())
    df.loc[mask, dev_col] = scores

# 3) Save back the full DataFrame
df.to_json(DEV_FILE, orient='records', lines=True)
print(f"Developability scores ready: {df[dev_col].notna().sum()} / {len(df)} sequences")

Developability scores ready: 64 / 64 sequences


In [17]:
# %% Prepare training matrices for GP models
# Convert embeddings from list back to array
X = np.vstack(df[embed_col].values).astype(np.float64)
Y = df[['y_low', 'y_medium', 'y_high']].values.astype(np.float64)

F = np.hstack([[FID_MAP[c]] * len(df) for c in ['y_low', 'y_medium', 'y_high']]).astype(np.float64)
X_rep = np.repeat(X, 3, axis=0)
Y_flat = Y.ravel()

mf_gp = MultiFidelityGP(X_rep, Y_flat, F, FID_MAP.values())
dev_gp = DevelopabilityGP(X, df[dev_col].values.astype(np.float64))


/Users/simonpritchard/miniconda3/envs/bayes_opt/lib/python3.10/site-packages/botorch/models/utils/assorted.py:270: InputDataWarning: Data (input features) is not contained to the unit cube. Please consider min-max scaling the input data.
  check_min_max_scaling(
/Users/simonpritchard/miniconda3/envs/bayes_opt/lib/python3.10/site-packages/botorch/models/utils/assorted.py:273: InputDataWarning: Data (outcome observations) is not standardized (std = tensor([0.], dtype=torch.float64), mean = tensor([0.], dtype=torch.float64)).Please consider scaling the input to zero mean and unit variance.
  check_standardization(Y=train_Y, raise_on_fail=raise_on_fail)


In [18]:
# %% Select initial seed sequence
best_idx = int(np.argmax(mf_gp.y))
seed_seq = df.sequence.iloc[best_idx // 3]
print('Seed sequence selected (index):', best_idx // 3)


Seed sequence selected (index): 54


In [19]:
# %% Acquisition function helper
def make_acq(cfg, mf_gp):
    y_best = mf_gp.y.max()
    if cfg.acq == 'ei':
        return lambda Xc: expected_improvement(Xc, mf_gp, y_best, cfg.xi)
    elif cfg.acq == 'ucb':
        return lambda Xc: upper_confidence_bound(Xc, mf_gp, cfg.kappa)
    elif cfg.acq == 'ts':
        return lambda Xc: thompson_sampling(Xc, mf_gp)
    else:
        raise ValueError(cfg.acq)


In [20]:
# build our embedding‐getter
interim_dir = Path("../data/interim/cd98_64_biophi.jsonl")
embed_col   = "pca_embed"

get_embedding = make_embedding_lookup(
    df,        # your DataFrame from earlier cells
    embed_col,
    cfg,       # your PipelineConfig instance
    interim_dir
)

In [21]:
@lru_cache(maxsize=200_000)
def _embed_cached(seq: str) -> np.ndarray:
    # lazy call into your existing PCA‑backed embed_single
    return embed_single(seq, cfg.embed_components)

@lru_cache(maxsize=200_000)
def cached_fitness(seq: str) -> float:
    emb = _embed_cached(seq).reshape(1, -1)
    return mf_gp.predict(emb)[0]

def score_batch(seqs: list[str]) -> np.ndarray:
    # if you ever want to evaluate many seqs at once, in one vectorized GP call:
    embs = np.vstack([_embed_cached(s) for s in seqs])
    return mf_gp.predict(embs)

In [ ]:
# --- 2) main BO loop ---
history = []
assert X.shape[1] == cfg.embed_components, (
    f"Expected X dim {cfg.embed_components}, got {X.shape[1]}"
)
print("asserted X")

for it in tqdm(range(cfg.n_iter), desc="BO iterations"):
    acq_fn = make_acq(cfg, mf_gp)
    print("made acquisition function")

    batch_seqs, batch_embs, batch_ys = [], [], []
    for _ in tqdm(range(cfg.batch_k), desc=f"Iter {it:03d} candidates", leave=True, position=1):
        # use the *cached* fitness function
        fitness_fn = cached_fitness
        print("initialized cached fitness function")

        # 1) propose sequences
        if cfg.seq_opt == "hc":
            cand = hill_climb(
                seed_seq,
                fitness_fn,
                local_k2_samples=cfg.seq_proposals or 1000,
                restarts=1,
            )
            print("selected a candidate with hill‑climb")
        elif cfg.seq_opt == "ga":
            cand = genetic_algorithm(
                [seed_seq],
                fitness_fn,
                pop_size=cfg.seq_proposals or 200,
            )
        elif cfg.seq_opt == "gs":
            cand = gibbs_sampling(
                seed_seq,
                fitness_fn,
                iters=5, #(cfg.seq_proposals or 1000) * 10
            )
        else:
            raise ValueError(f"Unknown seq_opt: {cfg.seq_opt}")

        # 2) dedupe & embed batch (cached)
        cand = list(dict.fromkeys(cand))
        print(f"here's the candidate set (deduped): {len(cand)} seqs")
        embs = np.vstack([_embed_cached(s) for s in cand])
        print("embedded batch with cache")

        # 3) acquisition + dev weighting
        f_acq    = acq_fn(embs)                # vectorized
        dev_mean = dev_gp.predict(embs)        # also vectorized
        combined = (1 - cfg.dev_weight) * f_acq + cfg.dev_weight * dev_mean
        idx      = int(np.argmax(combined))

        # 4) pick & “experiment”
        seq_next = cand[idx]
        x_next   = embs[idx].astype(np.float32)
        y_next   = -np.sum((x_next - 0.5)**2) + np.sin(5 * np.sum(x_next))

        # 5) update GP & history
        mf_gp.add_data(x_next, y_next)
        batch_seqs.append(seq_next)
        batch_embs.append(x_next)
        batch_ys.append(y_next)
        seed_seq = seq_next

    history.append({
        "iter":      it,
        "sequences": batch_seqs,
        "X":         np.vstack(batch_embs),
        "y":         np.array(batch_ys),
    })
    print(
        f"Iter {it:03d} | best_y {mf_gp.y.max():.3f} | "
        f"batch mean {np.mean(batch_ys):.3f}"
    )

# %% Inspect results
print(f"Optimization finished. Total evaluated points: {len(mf_gp.y)}")
best_id = int(np.argmax(mf_gp.y))
print(f'Best fitness: {mf_gp.y[best_id]:.3f}')

